In [10]:
import numpy as np
import pandas as pd

### Aufgabe 1
Format für products: Name, Preis, Gewicht

In [11]:
products = np.array(["Edelstein","Epoxidharz","Kupfer"])
profits = np.array([30, 10, 100])
weights = np.array([2, 1, 6])



In [12]:
# Jedes Szenario wird durchlaufen (BruteForce)
def check_constraints(ed, ku, w_total, u_total, w_max, u_max):
    return ku <= 2 * ed and w_total <= w_max and u_total <= u_max

def optimize(u_max, w_max):
    results = []
    u_max = u_max
    for ed in range(u_max):
        for ep in range(u_max):
            for ku in range(u_max):
                u_total = ed + ep + ku
                w_total = weights[0] * ed + weights[1] * ep + weights[2] * ku
                if check_constraints(ed, ku, w_total, u_total, w_max, u_max):
                    profit = profits[0] * ed + profits[1] * ep + profits[2] * ku
                    results.append({
                        "Gewinn": profit,
                        "Gewicht": w_total,
                        "Einheiten": u_total,
                        "Edelstein": ed,
                        "Epoxidharz": ep,
                        "Kupfer": ku
                    })

    return results


In [13]:
max_units = 15
max_weight = 56
optimized_products = optimize(max_units, max_weight)
df = pd.DataFrame(optimized_products)
# Optimized_Products wird nach Gewinn sortiert und es werden nur die "Top 10" angezeigt
sorted_df = df.sort_values(by="Gewinn", ascending=False).reset_index(drop=True).loc[:10]

In [14]:
print(sorted_df)

    Gewinn  Gewicht  Einheiten  Edelstein  Epoxidharz  Kupfer
0      920       56         12          4           0       8
1      910       56         14          7           0       7
2      900       56         15          6           2       7
3      890       55         14          6           1       7
4      880       54         13          6           0       7
5      880       55         15          5           3       7
6      870       54         14          5           2       7
7      870       54         15          9           0       6
8      860       53         13          5           1       7
9      860       54         15          4           4       7
10     850       53         14          4           3       7


## Nicht brute-force mit SciPy
- Wir gehen hier wieder von 0 aus, das heißt die Variablen von der BruteForce methode müssen vorher gecleared werden

In [2]:
from scipy.optimize import linprog
import numpy as np
import pandas as pd

In [5]:
products = np.array(["Edelstein", "Epoxidharz", "Kupfer"])
profits = np.array([30, 10, 100])
weights = np.array([2, 1, 6])

u_max = 15
w_max = 56

# Da scipy minimiert, müssen Gewinne negativ gesetzt werden!
c = -profits

# Bedingungen
A_ub = np.array([
    weights,
    np.ones(len(products)), # Summe der Einheiten
    [-2, 0, 1] # Kupfer <= 2 * Edelsteine
])
b_ub = np.array([w_max, u_max, 0])

bounds = [(0, None) for _ in products]

res = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method="highs")

if res.success:
    solution = np.floor(res.x)
    df = pd.DataFrame({
        "Gewinn": solution * profits,
        "Produkt": products,
        "Menge": solution.astype(int)
    })
    print(df)
    print(f"Gesamtgewinn: {np.dot(profits, solution)}")
else:
    print("Fehler: ", res.message)


   Gewinn     Produkt  Menge
0   120.0   Edelstein      4
1     0.0  Epoxidharz      0
2   800.0      Kupfer      8
Gesamtgewinn: 920.0
